# License Plate Recognition Training - YOLOv8

Training model untuk OCR karakter plat nomor di server.

**Dataset**: Indonesian License Plate Recognition Dataset (~1900 images)

**Classes**: 37 (0-9, A-Z)

**Model**: YOLOv8 (lebih akurat untuk server dengan resource cukup)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_REPO/license-plate-recognition-training.ipynb)

## 🚀 Google Colab Setup

In [ ]:
# Check if running on Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("✓ Running on Google Colab")
    # Check GPU
    !nvidia-smi
else:
    print("✓ Running locally")

## 1. Setup Environment

In [ ]:
# Install ultralytics (YOLOv8)
!pip install -q ultralytics kagglehub
!pip install -q opencv-python matplotlib pillow pyyaml tqdm pandas scikit-learn

In [ ]:
import os
import yaml
import shutil
from pathlib import Path
import glob
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import pandas as pd
from ultralytics import YOLO
import cv2
import kagglehub
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, confusion_matrix
from collections import Counter
import time

print("✓ Environment setup complete!")
print(f"Ultralytics version: {__import__('ultralytics').__version__}")

## 2. Download Dataset from Kaggle

In [ ]:
# Download Indonesian License Plate Recognition Dataset
# Note: Gunakan dataset yang sama atau dataset khusus untuk recognition jika ada
print("Downloading Indonesian License Plate Dataset from Kaggle...")
path = kagglehub.dataset_download("juanthomaswijaya/indonesian-license-plate-dataset")

print(f"✓ Dataset downloaded to: {path}")
DATASET_ROOT = Path(path)

# Cari folder Indonesian License Plate Recognition Dataset
# Adjust path based on actual dataset structure
recognition_dataset_candidates = list(DATASET_ROOT.glob('**/Indonesian License Plate Recognition Dataset'))
if recognition_dataset_candidates:
    DATASET_ROOT = recognition_dataset_candidates[0]
    print(f"Found recognition dataset at: {DATASET_ROOT}")
else:
    print("Recognition dataset not found in expected location.")
    print("Available directories:")
    for item in DATASET_ROOT.rglob('*'):
        if item.is_dir():
            print(f"  {item}")

## 3. Dataset Exploration

In [ ]:
# Dataset paths
IMAGES_DIR = DATASET_ROOT / 'images'
LABELS_DIR = DATASET_ROOT / 'labels'
classes_file = DATASET_ROOT / 'classes.names'

# Load class names
if classes_file.exists():
    with open(classes_file, 'r') as f:
        class_names = [line.strip() for line in f.readlines()]
else:
    # Default class names untuk Indonesian plates
    class_names = [str(i) for i in range(10)] + [chr(i) for i in range(ord('A'), ord('Z')+1)]
    print("⚠️  classes.names file not found, using default 0-9, A-Z")

print(f"Dataset root: {DATASET_ROOT.exists()}")
print(f"\nNumber of classes: {len(class_names)}")
print(f"Classes: {class_names}")

# Count images
train_images = list((IMAGES_DIR / 'train').glob('*.jpg')) + list((IMAGES_DIR / 'train').glob('*.png'))
val_images = list((IMAGES_DIR / 'val').glob('*.jpg')) + list((IMAGES_DIR / 'val').glob('*.png'))
test_images = list((IMAGES_DIR / 'test').glob('*.jpg')) + list((IMAGES_DIR / 'test').glob('*.png'))

print(f"\n📊 Dataset Statistics:")
print(f"{'='*40}")
print(f"Train images: {len(train_images):>6}")
print(f"Val images:   {len(val_images):>6}")
print(f"Test images:  {len(test_images):>6}")
print(f"{'='*40}")
print(f"Total:        {len(train_images) + len(val_images) + len(test_images):>6}")

In [ ]:
# Visualize samples with character annotations
def visualize_ocr_samples(image_paths, label_dir, class_names, num_samples=4):
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.ravel()
    
    samples = np.random.choice(image_paths, min(num_samples, len(image_paths)), replace=False)
    
    for idx, img_path in enumerate(samples):
        img = Image.open(img_path).convert('RGB')
        img_array = np.array(img)
        img_w, img_h = img.size
        
        # Find label file
        label_file = label_dir / (img_path.stem + '.txt')
        
        # Draw on image
        draw_img = img_array.copy()
        
        if label_file.exists():
            with open(label_file, 'r') as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    if len(parts) < 5:
                        continue
                    
                    class_id = int(parts[0])
                    x_center, y_center, width, height = map(float, parts[1:5])
                    
                    # Convert to pixels
                    x_center *= img_w
                    y_center *= img_h
                    width *= img_w
                    height *= img_h
                    
                    x1 = int(x_center - width / 2)
                    y1 = int(y_center - height / 2)
                    x2 = int(x_center + width / 2)
                    y2 = int(y_center + height / 2)
                    
                    # Draw rectangle
                    cv2.rectangle(draw_img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    
                    # Draw label
                    label = class_names[class_id] if class_id < len(class_names) else str(class_id)
                    cv2.putText(draw_img, label, (x1, y1-5), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
        
        axes[idx].imshow(draw_img)
        axes[idx].axis('off')
        axes[idx].set_title(f'{img_path.name}', fontsize=10)
    
    plt.tight_layout()
    plt.show()

# Visualize training samples
print("📸 Training Data Samples with Character Annotations:")
visualize_ocr_samples(train_images, LABELS_DIR / 'train', class_names)

In [ ]:
# Analyze class distribution
def count_class_distribution(label_dir):
    class_counts = Counter()
    label_files = list(label_dir.glob('*.txt'))
    
    for label_file in label_files:
        with open(label_file, 'r') as f:
            for line in f.readlines():
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    class_counts[class_id] += 1
    
    return class_counts

train_class_counts = count_class_distribution(LABELS_DIR / 'train')

# Plot distribution
fig, ax = plt.subplots(figsize=(16, 6))
classes = sorted(train_class_counts.keys())
counts = [train_class_counts[c] for c in classes]
labels = [class_names[c] if c < len(class_names) else str(c) for c in classes]

bars = ax.bar(labels, counts, color='steelblue', edgecolor='black')
ax.set_xlabel('Character Class', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.set_title('Character Distribution in Training Set', fontsize=14, weight='bold')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n📊 Class Distribution Statistics:")
print(f"{'='*40}")
print(f"Total characters: {sum(counts):,}")
print(f"Most common: {class_names[classes[counts.index(max(counts))]]} ({max(counts):,} instances)")
print(f"Least common: {class_names[classes[counts.index(min(counts))]]} ({min(counts):,} instances)")
print(f"Average per class: {sum(counts)/len(counts):.1f}")

## 4. Create YOLOv8 Dataset Configuration

In [ ]:
# Create dataset.yaml for YOLOv8
dataset_yaml = {
    'path': str(DATASET_ROOT.absolute()),
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': len(class_names),
    'names': class_names
}

yaml_path = 'license_plate_recognition.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_yaml, f, sort_keys=False)

print(f"✓ Dataset config saved to: {yaml_path}")
print("\nContent:")
with open(yaml_path, 'r') as f:
    content = f.read()
    # Show truncated version if too long
    lines = content.split('\n')
    if len(lines) > 15:
        print('\n'.join(lines[:10]))
        print(f"... ({len(lines)-15} more lines)")
        print('\n'.join(lines[-5:]))
    else:
        print(content)

## 5. Train YOLOv8 Model

In [ ]:
# Training configuration
MODEL = 'yolov8n'  # nano model (untuk balance speed & accuracy)
# Alternative: yolov8s, yolov8m untuk akurasi lebih tinggi

EPOCHS = 150
BATCH_SIZE = 16 if IN_COLAB else 8
IMG_SIZE = 640
PROJECT = 'runs/ocr'
NAME = 'license_plate_recognition'

print(f"🔧 Training Configuration:")
print(f"{'='*40}")
print(f"Model:       {MODEL}")
print(f"Epochs:      {EPOCHS}")
print(f"Batch size:  {BATCH_SIZE}")
print(f"Image size:  {IMG_SIZE}")
print(f"Classes:     {len(class_names)} (0-9, A-Z)")
print(f"Device:      {'GPU (Colab)' if IN_COLAB else 'CPU/GPU'}")
print(f"{'='*40}")

In [ ]:
# Initialize model
model = YOLO(f'{MODEL}.pt')

# Train
print("\n🚀 Starting training...\n")
results = model.train(
    data=yaml_path,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    project=PROJECT,
    name=NAME,
    patience=30,
    save=True,
    cache=True,
    plots=True,
    verbose=True
)

print("\n✓ Training complete!")

## 6. Comprehensive Model Evaluation

In [ ]:
# Load best model
best_model_path = Path(PROJECT) / NAME / 'weights' / 'best.pt'
model = YOLO(str(best_model_path))

print(f"✓ Loaded best model from: {best_model_path}")

In [ ]:
# Validate on test set
print("\n🧪 Testing on TEST SET...")
print("="*60)

metrics = model.val(
    data=yaml_path,
    split='test',
    imgsz=IMG_SIZE,
    plots=True,
    save_json=True,
    verbose=True
)

print("\n📊 TEST SET METRICS:")
print("="*60)
print(f"mAP@0.5:.......... {metrics.box.map50:.4f} ({metrics.box.map50*100:.2f}%)")
print(f"mAP@0.5:0.95:..... {metrics.box.map:.4f} ({metrics.box.map*100:.2f}%)")
print(f"Precision:........ {metrics.box.mp:.4f} ({metrics.box.mp*100:.2f}%)")
print(f"Recall:........... {metrics.box.mr:.4f} ({metrics.box.mr*100:.2f}%)")

# Calculate F1 Score
precision = metrics.box.mp
recall = metrics.box.mr
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
print(f"\n🎯 F1 SCORE:....... {f1:.4f} ({f1*100:.2f}%)")
print("="*60)

In [ ]:
# Display training results
from IPython.display import Image as IPImage, display

results_dir = Path(PROJECT) / NAME

print("\n📈 TRAINING VISUALIZATIONS")
print("="*60)

# Display training curves
if (results_dir / 'results.png').exists():
    print("\n1️⃣  Training Curves:")
    display(IPImage(filename=str(results_dir / 'results.png')))

# Display confusion matrix
conf_matrix_files = ['confusion_matrix_normalized.png', 'confusion_matrix.png']
for cf in conf_matrix_files:
    if (results_dir / cf).exists():
        print(f"\n2️⃣  Confusion Matrix:")
        display(IPImage(filename=str(results_dir / cf)))
        break

# Display PR curve
if (results_dir / 'PR_curve.png').exists():
    print("\n3️⃣  Precision-Recall Curve:")
    display(IPImage(filename=str(results_dir / 'PR_curve.png')))

# Display F1 curve
if (results_dir / 'F1_curve.png').exists():
    print("\n4️⃣  F1 Confidence Curve:")
    display(IPImage(filename=str(results_dir / 'F1_curve.png')))

## 7. Detailed Per-Class Evaluation

In [ ]:
# Per-class metrics analysis
# This provides detailed F1, Precision, Recall for each character

print("\n📋 PER-CLASS PERFORMANCE METRICS:")
print("="*80)

# Get per-class metrics if available
if hasattr(metrics.box, 'p') and hasattr(metrics.box, 'r'):
    per_class_precision = metrics.box.p
    per_class_recall = metrics.box.r
    
    # Create dataframe for better visualization
    metrics_df = pd.DataFrame({
        'Class': class_names[:len(per_class_precision)],
        'Precision': per_class_precision,
        'Recall': per_class_recall
    })
    
    # Calculate F1 per class
    metrics_df['F1'] = 2 * (metrics_df['Precision'] * metrics_df['Recall']) / \
                       (metrics_df['Precision'] + metrics_df['Recall'] + 1e-10)
    
    # Sort by F1 score
    metrics_df_sorted = metrics_df.sort_values('F1', ascending=False)
    
    print("\nTop 10 Best Performing Classes:")
    print(metrics_df_sorted.head(10).to_string(index=False))
    
    print("\n\nTop 10 Worst Performing Classes:")
    print(metrics_df_sorted.tail(10).to_string(index=False))
    
    # Overall statistics
    print("\n\n📊 Per-Class Statistics:")
    print("="*60)
    print(f"Average F1:.........{metrics_df['F1'].mean():.4f}")
    print(f"Std Dev F1:......... {metrics_df['F1'].std():.4f}")
    print(f"Best F1 (Class {metrics_df_sorted.iloc[0]['Class']}):..... {metrics_df_sorted.iloc[0]['F1']:.4f}")
    print(f"Worst F1 (Class {metrics_df_sorted.iloc[-1]['Class']}):..... {metrics_df_sorted.iloc[-1]['F1']:.4f}")
    print("="*60)
else:
    print("Per-class metrics not available in this YOLOv8 version.")

## 8. Test OCR on Sample Images

In [ ]:
# Function to reconstruct plate text from detections
def reconstruct_plate_text(results, img_width):
    """
    Reconstruct license plate text from character detections.
    Sort by x-coordinate (left to right)
    """
    boxes = results[0].boxes
    
    if len(boxes) == 0:
        return "", 0.0
    
    # Extract detections
    detections = []
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = float(box.conf[0])
        cls = int(box.cls[0])
        x_center = (x1 + x2) / 2
        detections.append((x_center, cls, conf))
    
    # Sort by x-coordinate (left to right)
    detections.sort(key=lambda x: x[0])
    
    # Build text
    text = ''.join([class_names[cls] for _, cls, _ in detections])
    avg_conf = np.mean([conf for _, _, conf in detections])
    
    return text, avg_conf

In [ ]:
# Test on random samples from test set
fig, axes = plt.subplots(3, 3, figsize=(18, 18))
axes = axes.ravel()

sample_imgs = np.random.choice(test_images, min(9, len(test_images)), replace=False)

correct_predictions = 0
total_predictions = 0

for idx, img_path in enumerate(sample_imgs):
    img = Image.open(img_path)
    img_w, img_h = img.size
    
    # Run inference
    results = model(img, conf=0.25, verbose=False)
    
    # Reconstruct text
    plate_text, confidence = reconstruct_plate_text(results, img_w)
    
    # Render predictions
    annotated = results[0].plot()
    annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    
    axes[idx].imshow(annotated_rgb)
    axes[idx].axis('off')
    
    title_color = 'green' if confidence > 0.7 else 'orange' if confidence > 0.5 else 'red'
    axes[idx].set_title(
        f'Predicted: "{plate_text}"\nConf: {confidence:.2f}', 
        fontsize=11, 
        fontweight='bold',
        color=title_color
    )
    
    total_predictions += 1
    if confidence > 0.7:
        correct_predictions += 1

plt.tight_layout()
plt.suptitle('Test Set OCR Results', fontsize=16, y=1.01, weight='bold')
plt.show()

print(f"\nHigh Confidence Predictions (> 0.7): {correct_predictions}/{total_predictions} ({correct_predictions/total_predictions*100:.1f}%)")

## 9. Inference Speed Benchmark

In [ ]:
# Benchmark inference speed
test_img = str(test_images[0])
img = Image.open(test_img)

# Warmup
print("Warming up model...")
for _ in range(10):
    _ = model(img, verbose=False)

# Benchmark
num_runs = 100
print(f"Running {num_runs} inference iterations...")
times = []
for _ in range(num_runs):
    start = time.time()
    results = model(img, verbose=False)
    times.append(time.time() - start)

times = np.array(times) * 1000  # Convert to ms

print("\n" + "="*60)
print("⚡ OCR INFERENCE SPEED BENCHMARK")
print("="*60)
print(f"Runs:...................... {num_runs}")
print(f"Mean:...................... {times.mean():.2f} ms")
print(f"Median:.................... {np.median(times):.2f} ms")
print(f"Std Dev:................... {times.std():.2f} ms")
print(f"Min:....................... {times.min():.2f} ms")
print(f"Max:....................... {times.max():.2f} ms")
print(f"FPS (mean):................ {1000/times.mean():.2f}")
print(f"\nThis is fast enough for server-side processing! ✓")
print("="*60)

# Plot distribution
plt.figure(figsize=(10, 5))
plt.hist(times, bins=30, color='lightcoral', edgecolor='black')
plt.axvline(times.mean(), color='red', linestyle='--', label=f'Mean: {times.mean():.2f} ms')
plt.axvline(np.median(times), color='green', linestyle='--', label=f'Median: {np.median(times):.2f} ms')
plt.xlabel('Inference Time (ms)')
plt.ylabel('Frequency')
plt.title('OCR Inference Time Distribution')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

## 10. Export Model for Deployment

In [ ]:
# Create export directory
export_dir = Path('../models/recognition') if not IN_COLAB else Path('/content/models_export/recognition')
export_dir.mkdir(parents=True, exist_ok=True)

print(f"✓ Export directory: {export_dir.absolute()}")

In [ ]:
# Export to ONNX
print("Exporting to ONNX format...")
onnx_path = model.export(format='onnx', imgsz=IMG_SIZE, simplify=True)
shutil.copy(onnx_path, export_dir / 'license_plate_recognition.onnx')
print(f"✓ ONNX model exported to: {export_dir / 'license_plate_recognition.onnx'}")
print(f"  Size: {os.path.getsize(export_dir / 'license_plate_recognition.onnx') / 1024 / 1024:.2f} MB")

In [ ]:
# Copy PyTorch weights
shutil.copy(best_model_path, export_dir / 'license_plate_recognition.pt')
print(f"✓ PyTorch weights saved to: {export_dir / 'license_plate_recognition.pt'}")
print(f"  Size: {os.path.getsize(export_dir / 'license_plate_recognition.pt') / 1024 / 1024:.2f} MB")

# Also save class names
with open(export_dir / 'classes.names', 'w') as f:
    f.write('\n'.join(class_names))
print(f"✓ Class names saved to: {export_dir / 'classes.names'}")

In [ ]:
# Download models (for Colab)
if IN_COLAB:
    from google.colab import files
    
    print("\n📥 Download models:")
    
    print("Downloading PyTorch model...")
    files.download(str(export_dir / 'license_plate_recognition.pt'))
    
    print("Downloading ONNX model...")
    files.download(str(export_dir / 'license_plate_recognition.onnx'))
    
    print("Downloading class names...")
    files.download(str(export_dir / 'classes.names'))

## 11. Summary & Final Report

In [ ]:
print("\n" + "="*70)
print("✅ LICENSE PLATE RECOGNITION (OCR) TRAINING COMPLETE")
print("="*70)
print(f"\n🔧 Model Configuration:")
print(f"  Model:................ {MODEL}")
print(f"  Training epochs:...... {EPOCHS}")
print(f"  Number of classes:.... {len(class_names)} (0-9, A-Z)")
print(f"  Best weights:......... {best_model_path}")
print(f"\n📦 Exported Models:")
print(f"  ONNX:................. {export_dir / 'license_plate_recognition.onnx'}")
print(f"  PyTorch:.............. {export_dir / 'license_plate_recognition.pt'}")
print(f"  Classes:.............. {export_dir / 'classes.names'}")
print(f"\n🎯 Final Test Set Metrics:")
print(f"  F1 Score:............. {f1:.4f} ({f1*100:.2f}%)")
print(f"  Precision:............ {precision:.4f} ({precision*100:.2f}%)")
print(f"  Recall:............... {recall:.4f} ({recall*100:.2f}%)")
print(f"  mAP@0.5:.............. {metrics.box.map50:.4f} ({metrics.box.map50*100:.2f}%)")
print(f"  mAP@0.5:0.95:......... {metrics.box.map:.4f} ({metrics.box.map*100:.2f}%)")
print(f"\n⚡ Performance:")
print(f"  Avg Inference:........ {times.mean():.2f} ms")
print(f"  FPS:.................. {1000/times.mean():.2f}")
print(f"\n📱 Next Steps:")
print(f"  1. Download models (PyTorch & ONNX)")
print(f"  2. Deploy to server (backend/python-service/)")
print(f"  3. Integrate dengan edge device (Raspberry Pi)")
print(f"  4. Test end-to-end parking entry system")
print(f"  5. Fine-tune on your specific camera setup if needed")
print("="*70)

# Save summary to file
summary_file = export_dir / 'training_summary.txt'
with open(summary_file, 'w') as f:
    f.write(f"License Plate Recognition Training Summary\n")
    f.write(f"="*60 + "\n")
    f.write(f"Model: {MODEL}\n")
    f.write(f"Epochs: {EPOCHS}\n")
    f.write(f"Classes: {len(class_names)}\n")
    f.write(f"\nTest Set Metrics:\n")
    f.write(f"F1 Score: {f1:.4f}\n")
    f.write(f"Precision: {precision:.4f}\n")
    f.write(f"Recall: {recall:.4f}\n")
    f.write(f"mAP@0.5: {metrics.box.map50:.4f}\n")
    f.write(f"mAP@0.5:0.95: {metrics.box.map:.4f}\n")
    f.write(f"\nInference Speed: {times.mean():.2f} ms\n")

print(f"\n✓ Training summary saved to: {summary_file}")